In [1]:
# imports
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd

# local
from env import get_db_url

# Exercises
This exercises use the cases, dept, and source tables from the 311_data on the Codeup MySQL server.



In [2]:
# Generate spark object to initialize a local Spark JVM process
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/05 15:21:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1. Read the case, department, and source data into their own spark dataframes.



In [3]:
case_df = pd.read_csv('case.csv')
case_df

,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,dept_division,service_request_type,SLA_days,case_status,source_id,request_address,council_district
0,1014127332,1/1/18 0:42,1/1/18 12:29,9/26/20 0:42,NO,-998.508762,YES,Field Operations,Stray Animal,999.000000,Closed,svcCRMLS,"2315 EL PASO ST, San Antonio, 78207",5
1,1014127333,1/1/18 0:46,1/3/18 8:11,1/5/18 8:30,NO,-2.012604,YES,Storm Water,Removal Of Obstruction,4.322222,Closed,svcCRMSS,"2215 GOLIAD RD, San Antonio, 78223",3
2,1014127334,1/1/18 0:48,1/2/18 7:57,1/5/18 8:30,NO,-3.022338,YES,Storm Water,Removal Of Obstruction,4.320729,Closed,svcCRMSS,"102 PALFREY ST W, San Antonio, 78223",3
3,1014127335,1/1/18 1:29,1/2/18 8:13,1/17/18 8:30,NO,-15.011481,YES,Code Enforcement,Front Or Side Yard Parking,16.291887,Closed,svcCRMSS,"114 LA GARDE ST, San Antonio, 78223",3
4,1014127336,1/1/18 1:34,1/1/18 13:29,1/1/18 4:34,YES,0.372164,YES,Field Operations,Animal Cruelty(Critical),0.125000,Closed,svcCRMSS,"734 CLEARVIEW DR, San Antonio, 78228",7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841699,1014127325,12/31/17 22:23,12/31/17 22:34,1/3/18 8:30,NO,-2.413681,YES,Signals,Traffic Signal Ops and Maintenance - Emergency,2.421319,Closed,120752,FLORES ST S and MARCH AVE,3
841700,1014127326,12/31/17 22:26,1/1/18 20:29,1/1/18 10:26,YES,0.418773,YES,Field Operations,Trapped/Confined Animal,0.500000,Closed,139345,"223 BRACKENRIDGE A, San Antonio, 78209",2
841701,1014127327,12/31/17 22:27,1/1/18 9:56,1/3/18 8:30,NO,-1.939688,YES,Signals,Traffic Signal Ops and Maintenance - Emergency,2.418275,Closed,CRM_Listener,COMMERCE ST W and ZARZAMORA N,5
841702,1014127329,12/31/17 22:41,3/19/18 20:33,1/5/18 22:41,YES,72.911493,YES,Field Operations,Animal Neglect,5.000000,Closed,139345,"5655 UTSA BLVD, San Antonio, 78249",8


In [4]:
case = spark.read.csv("case.csv", header=True, inferSchema=True)
case.show(5)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222| 

In [5]:
dept = spark.createDataFrame(pd.read_sql('select * from dept',get_db_url('311_data')))
dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



In [6]:
source = spark.createDataFrame(pd.read_sql('select * from source',get_db_url('311_data')))
source.show(5)

+-----+---------+----------------+
|index|source_id| source_username|
+-----+---------+----------------+
|    0|   100137|Merlene Blodgett|
|    1|   103582|     Carmen Cura|
|    2|   106463| Richard Sanchez|
|    3|   119403|  Betty De Hoyos|
|    4|   119555|  Socorro Quiara|
+-----+---------+----------------+
only showing top 5 rows



## 2. Let's see how writing to the local disk works in spark:



- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json


In [7]:
source.write.json('sources_json',mode='overwrite')
source.write.format('csv').mode('overwrite').option('header',True).save('sources_csv')

- Inspect your folder structure. What do you notice?


In [8]:
# both split into 7 parts with a success file and crc files for everything

## 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [9]:
case.schema

StructType([StructField('case_id', IntegerType(), True), StructField('case_opened_date', StringType(), True), StructField('case_closed_date', StringType(), True), StructField('SLA_due_date', StringType(), True), StructField('case_late', StringType(), True), StructField('num_days_late', DoubleType(), True), StructField('case_closed', StringType(), True), StructField('dept_division', StringType(), True), StructField('service_request_type', StringType(), True), StructField('SLA_days', DoubleType(), True), StructField('case_status', StringType(), True), StructField('source_id', StringType(), True), StructField('request_address', StringType(), True), StructField('council_district', IntegerType(), True)])

In [10]:
case.show(3)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222| 

In [11]:
# case_opened_date  str-date
# case_closed_date  str-date
# SLA_due_date      str-date
# case_late         str-bool
# case_closed       str-bool
# council_district  num-str max('SLA_due_date')
case = case.withColumn('case_closed', expr('case_closed=="YES"')
            ).withColumn('case_late', expr('case_late=="YES"')
            ).withColumn('council_district',col('council_district').cast('string')
            ).withColumn('case_opened_date',to_timestamp('case_opened_date','M/d/yy H:mm')
            ).withColumn('case_closed_date',to_timestamp('case_closed_date','M/d/yy H:mm')
            ).withColumn('SLA_due_date',to_timestamp('SLA_due_date','M/d/yy H:mm'))
case.show(3)

+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|   case_opened_date|   case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|    false| -998.5087616000001|       true|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|    false|-2.012604166999999

In [12]:
dept.schema

StructType([StructField('dept_division', StringType(), True), StructField('dept_name', StringType(), True), StructField('standardized_dept_name', StringType(), True), StructField('dept_subject_to_SLA', StringType(), True)])

In [13]:
dept.show(3)

+---------------+--------------------+----------------------+-------------------+
|  dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+--------------------+----------------------+-------------------+
|311 Call Center|    Customer Service|      Customer Service|                YES|
|          Brush|Solid Waste Manag...|           Solid Waste|                YES|
|Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
+---------------+--------------------+----------------------+-------------------+
only showing top 3 rows



In [14]:
# dept_subject_to_SLA   str-bool
dept = dept.withColumn('dept_subject_to_SLA',expr('dept_subject_to_SLA=="YES"'))
dept.show(3)

+---------------+--------------------+----------------------+-------------------+
|  dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+--------------------+----------------------+-------------------+
|311 Call Center|    Customer Service|      Customer Service|               true|
|          Brush|Solid Waste Manag...|           Solid Waste|               true|
|Clean and Green|Parks and Recreation|    Parks & Recreation|               true|
+---------------+--------------------+----------------------+-------------------+
only showing top 3 rows



In [15]:
source.schema

StructType([StructField('index', LongType(), True), StructField('source_id', StringType(), True), StructField('source_username', StringType(), True)])

In [16]:
source.show(3)

+-----+---------+----------------+
|index|source_id| source_username|
+-----+---------+----------------+
|    0|   100137|Merlene Blodgett|
|    1|   103582|     Carmen Cura|
|    2|   106463| Richard Sanchez|
+-----+---------+----------------+
only showing top 3 rows



## How old is the latest (in terms of days past SLA) currently open issue?

In [17]:
# 3 with same recently open date
case.select('SLA_days').where(expr('! case_closed')).sort(col('case_opened_date').desc()).show(3)

+--------+
|SLA_days|
+--------+
|    14.0|
|    14.0|
|     9.0|
+--------+
only showing top 3 rows



## How long has the oldest (in terms of days since opened) currently opened issue been open?

In [18]:
# case.select(datediff(current_timestamp(), 'case_opened_date').alias('case_age')).where(expr('! case_closed')).sort(col('case_opened_date')).show(1)
max_date = case.select(max('SLA_due_date')).first()[0]
case.select(datediff(lit(max_date), 'case_opened_date').alias('case_age')).where(expr('!case_closed')).show(1)

+--------+
|case_age|
+--------+
|    1606|
+--------+
only showing top 1 row



## How many Stray Animal cases are there?

In [19]:
case.where(expr('service_request_type=="Stray Animal"')).count()

26760

## How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [20]:
# case.where(expr('dept_division=="Field Operations"')).where(expr('service_request_type!="Officer Standby"')).count()
case.where(expr('dept_division=="Field Operations" and service_request_type!="Officer Standby"')).count()

113902

## Convert the council_district column to a string column.

In [21]:
# done earlier
case = case.withColumn('council_district',col('council_district').cast('string'))
case

DataFrame[case_id: int, case_opened_date: timestamp, case_closed_date: timestamp, SLA_due_date: timestamp, case_late: boolean, num_days_late: double, case_closed: boolean, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: string]

## Extract the year from the case_closed_date column.

In [22]:
case = case.withColumn('year_closed',year('case_closed_date'))
case.show(3)

+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-----------+
|   case_id|   case_opened_date|   case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|year_closed|
+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-----------+
|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|    false| -998.5087616000001|       true|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|       2018|
|1014127333|2018-01-01 00:46:00|2018-01-03 08:11:00|

## Convert num_days_late from days to hours in new columns num_hours_late.

In [23]:
case = case.withColumn('num_hours_late',expr('num_days_late*24'))
case.show(3)

+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-----------+-------------------+
|   case_id|   case_opened_date|   case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|year_closed|     num_hours_late|
+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-----------+-------------------+
|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|    false| -998.5087616000001|       true|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|     

## Join the case data with the source and department data.

In [24]:
case.show(3)

+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-----------+-------------------+
|   case_id|   case_opened_date|   case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|year_closed|     num_hours_late|
+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-----------+-------------------+
|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|    false| -998.5087616000001|       true|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|     

In [25]:
dept.show(3)

+---------------+--------------------+----------------------+-------------------+
|  dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+--------------------+----------------------+-------------------+
|311 Call Center|    Customer Service|      Customer Service|               true|
|          Brush|Solid Waste Manag...|           Solid Waste|               true|
|Clean and Green|Parks and Recreation|    Parks & Recreation|               true|
+---------------+--------------------+----------------------+-------------------+
only showing top 3 rows



In [26]:
source.show(3)

+-----+---------+----------------+
|index|source_id| source_username|
+-----+---------+----------------+
|    0|   100137|Merlene Blodgett|
|    1|   103582|     Carmen Cura|
|    2|   106463| Richard Sanchez|
+-----+---------+----------------+
only showing top 3 rows



In [27]:
df = (
    case
    # left join on dept_division
    .join(dept, 'dept_division', 'left')
    .join(source, 'source_id', 'left')
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .drop(source.source_id)
    .drop(case.source_id)
    .withColumnRenamed('standardized_dept_name', 'department')
    # convert last col to a boolean
    .withColumn('dept_subject_to_SLA', col('dept_subject_to_SLA')=="YES")
)

df.show(3)

+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+-----------+-------------------+--------------------+-------------------+-----+---------------+
|   case_id|   case_opened_date|   case_closed_date|       SLA_due_date|case_late|      num_days_late|case_closed|service_request_type|   SLA_days|case_status|     request_address|council_district|year_closed|     num_hours_late|          department|dept_subject_to_SLA|index|source_username|
+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+-----------+-------------------+--------------------+-------------------+-----+---------------+
|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|    false| -998.5087616000001|       true|       

## Are there any cases that do not have a request source?

In [28]:
# df.where(col('request_address').isNull()).count()
df.select([count(when(col('request_address').contains('None')\
                            | col('request_address').contains('NULL') \
                            | (col('request_address') == '' ) \
                            | col('request_address').isNull() \
                            | isnan('request_address')
                            , 'request_address')).alias('null_count')]).show()

+----------+
|null_count|
+----------+
|        31|
+----------+



## What are the top 10 service request types in terms of number of requests?

In [29]:
df.groupBy('service_request_type').agg(count('service_request_type').alias('num_requests')).sort(col('num_requests').desc()).show(10)

+--------------------+------------+
|service_request_type|num_requests|
+--------------------+------------+
|           No Pickup|       89210|
|Overgrown Yard/Trash|       66403|
|        Bandit Signs|       32968|
|        Damaged Cart|       31163|
|Front Or Side Yar...|       28920|
|        Stray Animal|       27361|
|Aggressive Animal...|       25492|
|Cart Exchange Req...|       22608|
|Junk Vehicle On P...|       21649|
|     Pot Hole Repair|       20827|
+--------------------+------------+
only showing top 10 rows



## What are the top 10 service request types in terms of average days late?


In [30]:
df.groupBy('service_request_type').agg(avg('num_days_late').alias('avg_days_late')).sort(col('avg_days_late').desc()).show(10)

+--------------------+------------------+
|service_request_type|     avg_days_late|
+--------------------+------------------+
|  Zoning: Junk Yards| 175.9563621042095|
|Labeling for Used...|162.43032902285717|
|Record Keeping of...|153.99724039428568|
|Signage Requied f...|151.63868055333333|
|Storage of Used M...|     142.112556415|
|Zoning: Recycle Yard|135.92851612479797|
|Donation Containe...|131.75610506358706|
|License Requied U...|128.79828704142858|
|Traffic Signal Gr...|101.79846062200002|
|           Complaint| 72.87050230311695|
+--------------------+------------------+
only showing top 10 rows



## Does number of days late depend on department?


In [31]:
df.groupBy('department').agg(avg('num_days_late').alias('avg_days_late')).sort(col('avg_days_late').desc()).show()

+--------------------+-------------------+
|          department|      avg_days_late|
+--------------------+-------------------+
|    Customer Service| 59.737091496300785|
|         Solid Waste|  -2.20005751367218|
|        Metro Health|    -4.911766979607|
|  Parks & Recreation| -5.251521960055134|
|Trans & Cap Impro...|-20.612837354052697|
|DSD/Code Enforcement| -38.36938892614464|
|Animal Care Services|-226.51783940550382|
|        City Council|               null|
+--------------------+-------------------+



## How do number of days late depend on department and request type?


In [32]:
df.groupBy('department','service_request_type').agg(avg('num_days_late').alias('avg_days_late')).sort(col('avg_days_late').desc()).show()

+--------------------+--------------------+------------------+
|          department|service_request_type|     avg_days_late|
+--------------------+--------------------+------------------+
|DSD/Code Enforcement|  Zoning: Junk Yards| 175.9563621042095|
|DSD/Code Enforcement|Labeling for Used...|162.43032902285717|
|DSD/Code Enforcement|Record Keeping of...|153.99724039428568|
|DSD/Code Enforcement|Signage Requied f...|151.63868055333333|
|DSD/Code Enforcement|Storage of Used M...|     142.112556415|
|DSD/Code Enforcement|Zoning: Recycle Yard|135.92851612479797|
|DSD/Code Enforcement|Donation Containe...|131.75610506358706|
|DSD/Code Enforcement|License Requied U...|128.79828704142858|
|Trans & Cap Impro...|Traffic Signal Gr...|101.79846062200002|
|    Customer Service|           Complaint| 72.87050230311695|
|DSD/Code Enforcement|             Vendors|   66.548098985078|
|  Parks & Recreation|Reservation Assis...|       66.03116319|
|DSD/Code Enforcement|   No Address Posted|59.875640733

## You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.

In [33]:
max_date = df.select(max('SLA_due_date')).first()[0]
max_date

datetime.datetime(2022, 5, 27, 15, 7)